<a href="https://colab.research.google.com/github/theDikshaSaxena/Recontextualization-Detection/blob/main/NarrativeDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch

In [2]:
%env CUDA_LAUNCH_BLOCKING=1
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
! pip install torchtext==0.10.1

import torch
device = torch.device("cuda")
torch.cuda.init()
import numpy as np
import time
import math

env: CUDA_LAUNCH_BLOCKING=1
ERROR: Could not find a version that satisfies the requirement torchtext==0.10.1 (from versions: 0.1.1, 0.2.0, 0.2.1, 0.2.3, 0.3.1, 0.4.0, 0.5.0, 0.6.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.1, 0.15.2)
ERROR: No matching distribution found for torchtext==0.10.1


RuntimeError: ignored

In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset

# Load the pretrained BERT model and tokenizer
model_name = "bert-base-uncased"  # You can also try "bert-base-cased" or other models
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Input paragraph you want to analyze
# paragraph = "I can't believe Russia would commit these war crimes so openly. Its time for the world to stop these attacks now before its to late."
paragraph = "Thursday, killing at least five people, hours ahead of the first face-to-face meeting since the start of the war between the Turkish and Ukrainian leaders.Moscow meanwhile denied it had deployed any heavy weapons at the Russian-controlled Zaporizhzhia nuclear power plant in southern Ukraine where a recent escalation in fighting has increased fears of a nuclear disaster.The head of the Kharkiv region Oleg Synegubov said Moscows forces had launched eight missiles from Russian territory at around 0430 local time (0130 GMT) striking across the city.Three people died, including a child. Eight people, including two children, were rescued, the emergency services said."

# Tokenize the paragraph
inputs = tokenizer.encode_plus(
    paragraph,
    None,
    add_special_tokens=True,
    max_length=128,
    pad_to_max_length=True,
    return_token_type_ids=True,
    return_attention_mask=True,
    return_tensors='pt',
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [5]:
# Perform sentiment analysis
model.eval()
with torch.no_grad():
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']
    outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)

In [6]:
# Interpret the results
prob_ukraine = probabilities[0][1].item()
prob_russia = probabilities[0][0].item()

if prob_ukraine > prob_russia:
    sentiment = "pro-Ukraine"
else:
    sentiment = "pro-Russia"

print("Paragraph Sentiment:", sentiment)
print("Probability of being pro-Ukraine:", prob_ukraine)
print("Probability of being pro-Russia:", prob_russia)

Paragraph Sentiment: pro-Russia
Probability of being pro-Ukraine: 0.43954840302467346
Probability of being pro-Russia: 0.5604516267776489
